<a href="https://colab.research.google.com/github/the-faisalahmed/Optimization/blob/main/Stable_Marriage_Problem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
import sys
import os

if 'google.colab' in sys.modules:
    !pip install idaes-pse --pre
    !idaes get-extensions --to ./bin
    os.environ['PATH'] += ':bin'

!pip install pyomo
from pyomo.environ import *
from pyomo.opt import SolverFactory
from pyomo.util.infeasible import log_infeasible_constraints
from pyomo.opt import SolverStatus
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import itertools
from pyomo.contrib.latex_printer import latex_printer


![](https://dmcommunity.org/wp-content/uploads/2024/06/stablemarriageproblem.png)

Given n men and n women, where each person has ranked all members of the opposite sex in order of preference, marry the men and women together such that there are no two people of opposite sex who would both rather have each other than their current partners. When there are no such pairs of people, the set of marriages is deemed stable.

![](https://dmcommunity.org/wp-content/uploads/2024/06/image-2.png)

Your task is to create a decision model capable to pair up a group of men and women so that the resulting marriages are stable.

In [2]:
singles = ['Adam', 'Bob', 'Charlie', 'Dave', 'Edgar','Alice', 'Barbara', 'Claire', 'Doris', 'Elsie']
matrix = {('Adam', 'Alice'): 5, ('Adam', 'Barbara'): 1, ('Adam', 'Claire'): 2, ('Adam', 'Doris'): 4, ('Adam', 'Elsie'): 3,
          ('Bob', 'Alice'): 4, ('Bob', 'Barbara'): 1, ('Bob', 'Claire'): 3, ('Bob', 'Doris'): 2, ('Bob', 'Elsie'): 5,
          ('Charlie', 'Alice'): 5, ('Charlie', 'Barbara'): 3, ('Charlie', 'Claire'): 2, ('Charlie', 'Doris'): 4, ('Charlie', 'Elsie'): 1,
          ('Dave', 'Alice'): 1, ('Dave', 'Barbara'): 5, ('Dave', 'Claire'): 4, ('Dave', 'Doris'): 3, ('Dave', 'Elsie'): 2,
          ('Edgar', 'Alice'): 4, ('Edgar', 'Barbara'): 3, ('Edgar', 'Claire'): 2, ('Edgar', 'Doris'): 1, ('Edgar', 'Elsie'): 5,
          ('Alice', 'Adam'): 5, ('Alice', 'Bob'): 1, ('Alice', 'Charlie'): 2, ('Alice', 'Dave'): 4, ('Alice', 'Edgar'): 3,
          ('Barbara', 'Adam'): 4, ('Barbara', 'Bob'): 1, ('Barbara', 'Charlie'): 3, ('Barbara', 'Dave'): 2, ('Barbara', 'Edgar'): 5,
          ('Claire', 'Adam'): 5, ('Claire', 'Bob'): 3, ('Claire', 'Charlie'): 2, ('Claire', 'Dave'): 4, ('Claire', 'Edgar'): 1,
          ('Doris', 'Adam'): 1, ('Doris', 'Bob'): 5, ('Doris', 'Charlie'): 4, ('Doris', 'Dave'): 3, ('Doris', 'Edgar'): 2,
          ('Elsie', 'Adam'): 4, ('Elsie', 'Bob'): 3, ('Elsie', 'Charlie'): 2, ('Elsie', 'Dave'): 1, ('Elsie', 'Edgar'): 5
          }

In [3]:
model = ConcreteModel()

women = ['Alice', 'Barbara', 'Claire', 'Doris', 'Elsie']
men = ['Adam', 'Bob', 'Charlie', 'Dave', 'Edgar']

model.V = list(itertools.permutations(singles,2))

model.P = Param(model.V, initialize = matrix)

model.X = Var(model.V, within = Binary)

# Each man should be matched with one woman

def con1_rule(model,i):
  return sum(model.X[(i,j)] for j in women) == 1
model.con1 = Constraint(men,rule = con1_rule)

# Each woman should be matched with one man

def con2_rule(model,i):
  return sum(model.X[(i,j)] for j in men) == 1
model.con2 = Constraint(women,rule = con2_rule)

# Couples that choose each other are connected

model.con3 = ConstraintList()

for i in men:
  for j in women:
    model.con3.add(expr = model.X[i,j] == model.X[j,i])

# Maximizing square values of couples that are matched while penalizing

def obj_rule(model):
  return sum((model.P[i,j]*model.X[i,j] + model.P[j,i]*model.X[j,i])**2 - \
   (model.P[i,j]*model.X[i,j] - model.P[j,i]*model.X[j,i]) for i in men for j in women)
model.obj = Objective(rule = obj_rule, sense = maximize)

In [4]:
# Solve model
opt = SolverFactory('bonmin')
result = opt.solve(model)

if (result.solver.status == SolverStatus.ok) and \
    (result.solver.termination_condition == TerminationCondition.optimal):
    # Do something when the solution in optimal and feasible
    print('Solution is Optimal')
elif (result.solver.termination_condition == TerminationCondition.infeasible):
    # Do something when model in infeasible
    print('Solution is Infeasible')
else:
        # Something else is wrong
    print("Solver Status:",  result.solver.status)

# Solve time
print('Solve Time: ', result.solver.wallclock_time)

# Objective Value
print('Objective value: ', value(model.obj))

Solution is Optimal
Solve Time:  <undefined>
Objective value:  352.0


In [5]:
couples = []
for (i,j) in model.X.extract_values().keys():
  if model.X.extract_values()[i,j] == 1:
    if i not in couples and j not in couples:
      print("{0} ({1}) and {2} ({3}) are a couple.".format(i,matrix[i,j],j,matrix[j,i]))
      couples.append(i and j)

Adam (5) and Alice (5) are a couple.
Bob (2) and Doris (5) are a couple.
Charlie (3) and Barbara (3) are a couple.
Dave (4) and Claire (4) are a couple.
Edgar (5) and Elsie (5) are a couple.
